In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库

In [2]:
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['day'] = data['context_timestamp_tmp'].dt.day
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    # del data['predict_property_1']
    # del data['predict_property_2']
#     del data['shop_score_service']
#     del data['shop_score_description']
#     del data['shop_review_positive_rate']
#     del data['shop_score_delivery']
    return data

In [3]:
data = pd.read_table('train.txt',sep=' ',index_col=False)
data2 = pd.read_table('test.txt',sep=' ',index_col=False) 
data3 = pd.read_table('test1.txt',sep=' ',index_col=False) 

In [4]:
data = pre_process(data)
data2 = pre_process(data2)
data3 = pre_process(data3)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [5]:
data[['user_id','day','is_trade']].groupby(['user_id','day']).sum().reset_index()

,user_id,day,is_trade
0,1523553916360,1,0
1,1523553916360,4,0
2,5742370312099,1,0
3,6731361713351,3,0
4,8455785307263,6,0
5,9451887501191,7,0
6,12217660235260,31,0
7,15213145006378,3,0
8,15213145006378,6,0
9,22634503310052,4,0


In [6]:
data[['user_id','day','is_trade']].groupby(['user_id','day']).count()>10

is_trade
user_id             day          
1523553916360       1       False
                    4       False
5742370312099       1       False
6731361713351       3       False
8455785307263       6       False
9451887501191       7       False
12217660235260      31      False
15213145006378      3       False
                    6       False
22634503310052      4       False
                    6       False
25684305694756      7       False
38014406741475      6       False
40480049735387      2       False
                    7       False
46479832058031      2       False
                    3       False
                    31      False
48857306404523      4       False
50153023443529      2       False
                    6       False
                    7       False
50156198040329      2       False
                    5       False
                    31      False
52161064826969      2       False
                    5       False
56829289324535      3       False
59640370843066      3       False
59919322738423      31      False
...                           ...
9223325323436067020 31      False
9223328961874648693 31      False
9223333305889619135 1       False
                    3        True
                    31      False
9223335771334020529 1       False
                    4       False
                    7       False
9223338824192191693 6       False
9223339674432588374 2       False
9223347096934779058 2       False
9223351358419350988 3       False
9223351764270311555 5       False
9223352063517690707 2       False
                    4       False
9223354933755916779 31      False
9223356530868126408 2       False
                    7       False
                    31      False
9223357827590849236 1       False
                    6       False
9223358947751079260 3       False
9223361258074308080 6       False
9223363919042445410 1       False
9223364994406019376 3       False
                    4       False
9223366924189890142 2       False
9223371754065880392 3       False
                    6       False
                    7       False

[4190641 rows x 1 columns]

In [7]:
all_data = data.append(data2)

In [8]:
all_data = all_data.append(data3)

In [9]:
user_click = all_data[['user_id','day','instance_id']].groupby(['user_id','day']).count()

In [10]:
user_click = user_click.rename(columns=({'instance_id':'day_click'}))

In [11]:
user_click = user_click.reset_index()

In [12]:
user_click

,user_id,day,day_click
0,1523553916360,1,3
1,1523553916360,4,5
2,5742370312099,1,2
3,6731361713351,3,1
4,8455785307263,6,4
5,9451887501191,7,1
6,12217660235260,31,2
7,12885223301783,7,1
8,15213145006378,3,1
9,15213145006378,6,1


In [13]:
userclickdata = pd.merge(data,user_click,on=['user_id','day'],how='left')

In [14]:
userclickdata2 = pd.merge(data2,user_click,on=['user_id','day'],how='left')

In [15]:
userclickdata3 = pd.merge(data3,user_click,on=['user_id','day'],how='left')

In [16]:
userclickdata[['instance_id','day_click']].to_csv('train_day_click.csv',index=False) 

In [17]:
userclickdata2[['instance_id','day_click']].to_csv('test_day_click.csv',index=False) 

In [18]:
userclickdata3[['instance_id','day_click']].to_csv('test1_day_click.csv',index=False) 

In [19]:
userclickdata3.shape

(1209768, 34)